In [1]:
import tensorflow as tf
import keras as kr
from keras.preprocessing.image import ImageDataGenerator


C:\Users\Anubha\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [18]:
cd ..

C:\Users\Anubha\Desktop


In [2]:
train_data_folder=['character_1_ka','character_2_kha','character_3_ga','character_4_gha','character_5_kna']
Label = ['Ka','Kha','Ga','Gha','Kna']
Labels = [0,1,2,3,4]

In [19]:

    
def load_photos(train_data_folder,Desktop):
    i=0
    y=[]
    images=[]
    for directory in train_data_folder:
        y_val=Labels[i]
        
        for name in listdir(Desktop+'/'+ directory):
            # load an image from file
            filename = Desktop+'/'+directory + '/' + name
            image = load_img(filename) #target_size=(32, 32,1))
            # convert the image pixels to a numpy array
            image = img_to_array(image)
            # reshape data for the model
            image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
            # prepare the image for the VGG model
            image = preprocess_input(image)

            images.append(image)
            y.append(y_val)
        i=i+1;
    return images,y

# load images
Train_Desktop = 'Train_val'
images,y = load_photos(train_data_folder,Train_Desktop)
print('Loaded Images: %d' % len(images))

Loaded Images: 8500


In [4]:
cd Desktop\Train_val

C:\Users\Anubha\Desktop\Train_val


In [6]:
import numpy as np
x_train = np.asarray(images)
y_train = np.asarray(y)

In [7]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
y_train=y_train.reshape(8500,1)
onehot_encoded = onehot_encoder.fit_transform(y_train)

In [8]:
datagen = ImageDataGenerator(
featurewise_center=True,
featurewise_std_normalization=True,
rotation_range=20,
width_shift_range=0.2,
height_shift_range=0.2)



In [11]:
datagen.fit(x_train)

In [10]:
input_shape = (32, 32, 3)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')


In [13]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
# Creating a Sequential Model and adding the layers
model = Sequential()
model.add(Conv2D(128, kernel_size=(3,3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(5,activation=tf.nn.softmax))

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 128)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 28800)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3686528   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645       
Total params: 3,690,757
Trainable params: 3,690,757
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [16]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=32)

Epoch 1/32
266/265 [==============================] - 109s 411ms/step - loss: 1.0807 - acc: 0.5807
Epoch 2/32
266/265 [==============================] - 104s 390ms/step - loss: 0.6725 - acc: 0.7594
Epoch 3/32
266/265 [==============================] - 105s 395ms/step - loss: 0.5577 - acc: 0.8079
Epoch 4/32
266/265 [==============================] - 106s 398ms/step - loss: 0.5061 - acc: 0.8269
Epoch 5/32
266/265 [==============================] - 105s 396ms/step - loss: 0.4585 - acc: 0.8501
Epoch 6/32
266/265 [==============================] - 105s 395ms/step - loss: 0.4402 - acc: 0.8541
Epoch 7/32
266/265 [==============================] - 107s 402ms/step - loss: 0.4162 - acc: 0.8667
Epoch 8/32
266/265 [==============================] - 106s 397ms/step - loss: 0.3842 - acc: 0.8726
Epoch 9/32
266/265 [==============================] - 108s 405ms/step - loss: 0.3583 - acc: 0.8756
Epoch 10/32
266/265 [==============================] - 106s 398ms/step - loss: 0.3845 - acc: 0.8769
Epoch 11/

In [21]:
Desktop_test = 'Test'
images_test,y_test = load_photos(train_data_folder,Desktop_test)
print('Loaded Images Test: %d' % len(images_test))
import numpy as np
x_test = np.asarray(images_test)
y_test = np.asarray(y_test)
x_test = x_test / 255.0
datagen.fit(x_test)

Loaded Images Test: 1500


In [23]:
score_ADAM = model.evaluate_generator(datagen.flow(x_test, y_test, batch_size=32),steps=len(x_test)/32 )
print ("Accuracy with ADAM for Model 0 : " , score_ADAM[1])

Accuracy with ADAM for Model 0 :  0.9586666666666667
